In [1]:
import pandas as pd
import numpy as np
import requests
from ydata_profiling import ProfileReport

In [2]:
# Obtain github paths for data downloads
examples_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_examples.parquet"
products_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_products.parquet"

# Download examples with query-product mappings
examples_response = requests.get(examples_url)
with open("../data/raw/examples.parquet", "wb") as f:
    f.write(examples_response.content)
del examples_response # Free up memory before next download

# Download products with product details  
products_response = requests.get(products_url)
with open("../data/raw/products.parquet", "wb") as f:
    f.write(products_response.content)
del products_response # Free up memory from large download

In [2]:
# Load datasets
df_examples = pd.read_parquet("../data/raw/examples.parquet")
df_products = pd.read_parquet("../data/raw/products.parquet")

# Merge on product_locale and product_id, per github repo instructions
df_examples_products = pd.merge(
    df_examples, 
    df_products,
    how='left',
    left_on=['product_locale', 'product_id'],
    right_on=['product_locale', 'product_id']
)

# Preliminary look at the merged dataset
print(f"Dataset shape: {df_examples_products.shape}")
df_examples_products.head()

Dataset shape: (2621288, 14)


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White


In [3]:
# Filter for task 1 and target criteria
df_task1 = df_examples_products[df_examples_products["small_version"] == 1]

# Look at the down-sized small version dataset
print(f"Dataset small_version shape: {df_task1.shape}")

# Apply filters for US locale and Exact matches only
df_filtered = df_task1[
    (df_task1['product_locale'] == 'us') & 
    (df_task1['esci_label'] == 'E')
]

# Look at the further filtered dataset
print(f"Dataset filtered shape: {df_filtered.shape}")
print(f"Unique queries: {df_filtered['query'].nunique()}")

# Another quick look at the data
df_filtered.head()

Dataset small_version shape: (1118011, 14)
Dataset filtered shape: (261527, 14)
Unique queries: 29843


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
17,17,!awnmower tires without rims,1,B08L3B9B9P,us,E,1,1,train,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,Please check your existing tire Sidewall for t...,MaxAuto,None
20,20,!awnmower tires without rims,1,B07C1WZG12,us,E,1,1,train,(Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Whe...,No fuss. Just take off your old assembly and r...,Tire size:15x6.00-6 Ply: 4 Tubeless\n6x4.5 Whe...,Antego Tire & Wheel,Husqvarna Silver
21,21,!awnmower tires without rims,1,B077QMNXTS,us,E,1,1,train,MaxAuto 2 Pcs 16x6.50-8 Lawn Mower Tire for Ga...,<br>Tire Specifications:<br> 1. Material: Rubb...,"Set of 2 16X6.50-8, 16x6.50x8, 16-6.50-8 Lawn ...",MaxAuto,Black
23,23,!awnmower tires without rims,1,B06XX6BM2R,us,E,1,1,train,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assem...",None,Tire: 2 pack 15x6. 00-6 tube-type turf SAVER t...,MARASTAR,None
26,26,!awnmower tires without rims,1,B0089RNSNM,us,E,1,1,train,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Set of 2 Honda OEM Rear Wheels\nReplaces 42710...,Honda,None


* Notice '!' exclamation point instead of 'l' for the first letter in lawnmower for query column.
  * Reiterates that text cleanup likely but maybe need to allow typos for user-generated queries

In [4]:
# Further look at the data before down-sampling
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261527 entries, 17 to 2618569
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   example_id            261527 non-null  int64 
 1   query                 261527 non-null  object
 2   query_id              261527 non-null  int64 
 3   product_id            261527 non-null  object
 4   product_locale        261527 non-null  object
 5   esci_label            261527 non-null  object
 6   small_version         261527 non-null  int64 
 7   large_version         261527 non-null  int64 
 8   split                 261527 non-null  object
 9   product_title         261527 non-null  object
 10  product_description   134067 non-null  object
 11  product_bullet_point  231366 non-null  object
 12  product_brand         249806 non-null  object
 13  product_color         184355 non-null  object
dtypes: int64(4), object(10)
memory usage: 29.9+ MB


* Many missing values for the 'product_' related columns:
  * ~51% for 'product_description' are missing!
  * ~11% for 'product_bullet_point' are missing
  * ~4% for 'product_brand' are missing
  * ~29% for 'product_color' are missing
* Some missing information may be likely for a product distributor, such as Amazon, if not provided by the manufacturer.
* However, distributors may likely try to obtain all product info before listing:
  * May be worth exploring if there is a way to drop null values before creating sample dataset

In [9]:
# Save current filtered dataset
df_filtered.to_parquet("../data/processed/df_filtered.parquet")

## Checkpoint 1

In [10]:
# Checkpoint to reload filtered dataset
df_filtered = pd.read_parquet("../data/processed/df_filtered.parquet")

In [5]:
# Drop null values for the 'product_' related columns
df_dropped = df_filtered.dropna(subset=['product_description', 'product_bullet_point', 'product_brand', 'product_color']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93296 entries, 0 to 93295
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            93296 non-null  int64 
 1   query                 93296 non-null  object
 2   query_id              93296 non-null  int64 
 3   product_id            93296 non-null  object
 4   product_locale        93296 non-null  object
 5   esci_label            93296 non-null  object
 6   small_version         93296 non-null  int64 
 7   large_version         93296 non-null  int64 
 8   split                 93296 non-null  object
 9   product_title         93296 non-null  object
 10  product_description   93296 non-null  object
 11  product_bullet_point  93296 non-null  object
 12  product_brand         93296 non-null  object
 13  product_color         93296 non-null  object
dtypes: int64(4), object(10)
memory usage: 10.0+ MB


* Full dataset with no missing values has ~93k records:
  * Try ~500 row / 50 unique query sample dataset creation

In [8]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_dropped['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_dropped[df_dropped['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 210 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,117965,6 acrylic brushes for nails,4839,B087CJ6ZVV,us,E,1,1,test,"Yolife Acrylic Nail Art Brush, 100% Pure Kolin...",<b>Package Content: </b><p>1* Red Wooden Nail ...,❤️【Ingenious Design】100% Pure Kolinsky Sable N...,Yolife,Wood
1,117966,6 acrylic brushes for nails,4839,B083ZMG2ZX,us,E,1,1,test,Pana USA Acrylic Nail Brush100% Pure Kolinsky ...,PANA Finest Kolinsky Hair Professional Acrylic...,[Design] Size 6 Brush Handle | A smooth and st...,PANA,Mahogany
2,117967,6 acrylic brushes for nails,4839,B07Y55F8HH,us,E,1,1,test,1PCS Eval 100% Kolinsky Sable hair Brushes Acr...,Eval Crimped Shaped Kolinsky Sable Brushes #6 ...,★HIGH QUALITY HAIR:Brushes made of Kolinsky sa...,Eval,Purple
3,137801,a 6 motels,5780,B07MTMP7D5,us,E,1,1,train,"Evocel Galaxy A6 (2018) Case, [Explorer Series...",The Evocel Explorer Series Pro is a phone case...,"[Dependable Protection] Heavy duty, dual layer...",Evocel,Blue
4,137803,a 6 motels,5780,B0796M8RPJ,us,E,1,1,train,Aunifun Replacement Filter & Side Brush Kit fo...,<b>Fit for</b> ILIFE A6 A4 A4s Robot Vacuum Cl...,★Fit for ILIFE A6 A4 A4s Robot Vacuum Cleaner....,Aunifun,Replacement Filter & Side Brush Kit for ILIFE ...


* Only able to sample to 210 rows from non-null ~93k dataset:
  * Likely need to accept some missing 'product_' values and develop imputation strategy
* Turn down-sampling into defined function and try different ways to handle missing values
  * Include original non-null down-sampling for good measure

In [7]:
def iterative_downsample(
    df: pd.DataFrame,
    subset_columns: list,
    target_rows: int = 500,
    target_queries: int = 50,
    random_seed: int = 2025
):
    np.random.seed(random_seed) # set random seed for reproducibility (like to use the year)
    
    def downsample(df_input):
        """Downsample df_input to the target number of queries and rows."""
        unique_queries = df_input['query'].unique()
        # Sample 50 queries randomly
        n_queries = min(target_queries, len(unique_queries))
        sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)
        # Get all products for these queries
        df_sample = df_input[df_input['query'].isin(sample_queries)].copy()
        
        # If too many rows, sample proportionally per query
        if df_sample.shape[0] > target_rows:
            target_per_query = max(1, target_rows // n_queries)
            df_sample = df_sample.groupby('query').apply(
                lambda x: x.sample(n=min(len(x), target_per_query), random_state=random_seed)
            ).reset_index(drop=True)
        else:
            df_sample = df_sample.reset_index(drop=True)

        return df_sample

    # Track results
    results = {}

    # Start with dropping all nulls from all specified columns
    current_cols = subset_columns.copy()
    df_dropped = df.dropna(subset=current_cols).reset_index(drop=True)
    df_sample = downsample(df_dropped)
    results['no_nulls'] = df_sample
    print(f"Sample with no nulls: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

    # Check what results would be for allowing nulls for each column, one at a time
    for col in subset_columns:
        current_cols = [c for c in subset_columns if c != col]
        df_partial = df.dropna(subset=current_cols).reset_index(drop=True)
        df_sample = downsample(df_partial)
        results[f"{col}_nulls_allowed"] = df_sample
        print(f"Sample with {col} nulls: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

    return results

In [8]:
subset_cols = ['product_description', 'product_bullet_point', 'product_brand', 'product_color']

sampled_dfs = iterative_downsample(df_filtered, subset_columns=subset_cols)

Sample with no nulls: 210 rows, 50 queries
Sample with product_description nulls: 294 rows, 50 queries
Sample with product_bullet_point nulls: 224 rows, 50 queries
Sample with product_brand nulls: 256 rows, 50 queries
Sample with product_color nulls: 246 rows, 50 queries


* Decide to allow null values as representative of real world problem
  * Define function to down-sample and coerce to row and query targets

In [9]:
def create_sample_dataset(
    df: pd.DataFrame,
    target_queries: int = 50,
    target_rows: int = 500,
    random_seed: int = 2025, 
    prioritize='rows', 
    max_queries=100
    ):
    """
    Create a sample dataset to meet both row and query targets.
    Allow to prioritize rows or queries, with preference for rows.
    Allow to set max queries to sample when prioritizing rows.
    """
    np.random.seed(random_seed)
    
    # Get query-level statistics to help with sampling
    query_counts = df['query'].value_counts()
    unique_queries = query_counts.index.tolist()
    print(f"Total unique queries available: {len(unique_queries)}")
    print(f"Query product counts - Min: {query_counts.min()}, Max: {query_counts.max()}, Mean: {query_counts.mean():.1f}")
    
    if prioritize == 'rows':
        print(f"\nPrioritizing hitting {target_rows} rows target...")
        
        # Strategy: Start with target_queries, increase if needed to hit row target
        current_queries = min(target_queries, len(unique_queries))
        
        for attempt_queries in range(current_queries, min(max_queries + 1, len(unique_queries) + 1)):
            # Sample queries
            sample_queries = np.random.choice(unique_queries, size=attempt_queries, replace=False)
            df_query_filtered = df[df['query'].isin(sample_queries)].copy()
            
            print(f"  Trying {attempt_queries} queries: {df_query_filtered.shape[0]} total rows")
            
            if df_query_filtered.shape[0] >= target_rows:
                print(f"  * Found {attempt_queries} queries with {df_query_filtered.shape[0]} rows (>= {target_rows})")
                
                # Now sample down to exactly target_rows
                if df_query_filtered.shape[0] > target_rows:
                    df_sample = df_query_filtered.sample(n=target_rows, random_state=random_seed).reset_index(drop=True)
                else:
                    df_sample = df_query_filtered.reset_index(drop=True)
                
                break
        else:
            # If we couldn't hit target_rows even with max_queries, use what we have
            print(f"  Could not reach {target_rows} rows even with {max_queries} queries")
            print(f"  Using all {df_query_filtered.shape[0]} rows from {attempt_queries} queries")
            df_sample = df_query_filtered.reset_index(drop=True)
    
    else:  # prioritize == 'queries'
        print(f"\nPrioritizing hitting {target_queries} queries target...")
        
        # Original strategy: sample exactly target_queries, then work with available rows
        n_queries = min(target_queries, len(unique_queries))
        sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)
        df_query_filtered = df[df['query'].isin(sample_queries)].copy()
        
        print(f"Sampled {n_queries} queries with {df_query_filtered.shape[0]} total rows")
        
        if df_query_filtered.shape[0] > target_rows:
            # Sample down proportionally per query
            rows_per_query = target_rows // n_queries
            remainder = target_rows % n_queries
            
            sampled_dfs = []
            queries_list = list(sample_queries)
            
            for i, query in enumerate(queries_list):
                query_df = df_query_filtered[df_query_filtered['query'] == query]
                n_rows_for_query = rows_per_query + (1 if i < remainder else 0)
                n_rows_for_query = min(n_rows_for_query, len(query_df))
                
                if n_rows_for_query > 0:
                    sampled_query_df = query_df.sample(n=n_rows_for_query, random_state=random_seed+i)
                    sampled_dfs.append(sampled_query_df)
            
            df_sample = pd.concat(sampled_dfs, ignore_index=True)
        else:
            df_sample = df_query_filtered.reset_index(drop=True)
    
    print(f"\nFinal sample dataset: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} unique queries")
    
    # Show missing value counts for product columns
    product_cols = ['product_description', 'product_bullet_point', 'product_brand', 'product_color']
    print(f"\nMissing values in product columns:")
    for col in product_cols:
        if col in df_sample.columns:
            missing_count = df_sample[col].isna().sum()
            missing_pct = (missing_count / len(df_sample)) * 100
            print(f"  {col}: {missing_count} ({missing_pct:.1f}%)")

    # Return both the sampled down and unsampled datasets    
    return df_sample, df_query_filtered

In [10]:
# Try the down-sampling prioritizing rows to hit the 500 target
df_sampled_rows, df_query_filtered_rows = create_sample_dataset(
    df_filtered, 
    target_queries=50, 
    target_rows=500, 
    random_seed=2025, 
    prioritize='rows',
    max_queries=100
)

Total unique queries available: 29843
Query product counts - Min: 1, Max: 81, Mean: 8.8

Prioritizing hitting 500 rows target...
  Trying 50 queries: 494 total rows
  Trying 51 queries: 528 total rows
  * Found 51 queries with 528 rows (>= 500)

Final sample dataset: 500 rows, 51 unique queries

Missing values in product columns:
  product_description: 247 (49.4%)
  product_bullet_point: 44 (8.8%)
  product_brand: 14 (2.8%)
  product_color: 138 (27.6%)


In [11]:
# Try the down-sampling prioritizing queries to hit the 50 target
df_sampled_queries, df_query_filtered_queries = create_sample_dataset(
    df_filtered, 
    target_queries=50, 
    target_rows=500, 
    random_seed=2025, 
    prioritize='queries'
)

Total unique queries available: 29843
Query product counts - Min: 1, Max: 81, Mean: 8.8

Prioritizing hitting 50 queries target...
Sampled 50 queries with 494 total rows

Final sample dataset: 494 rows, 50 unique queries

Missing values in product columns:
  product_description: 245 (49.6%)
  product_bullet_point: 102 (20.6%)
  product_brand: 47 (9.5%)
  product_color: 197 (39.9%)


* Prioritizing rows resulted in dataframes meeting required dataset sample sizes but prioritizing queries did not
* Move forward with exploring row-prioritized dataframes:
  *  Choose to accept slightly larger amount of rows or not through exploration

In [12]:
df_sampled_rows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            500 non-null    int64 
 1   query                 500 non-null    object
 2   query_id              500 non-null    int64 
 3   product_id            500 non-null    object
 4   product_locale        500 non-null    object
 5   esci_label            500 non-null    object
 6   small_version         500 non-null    int64 
 7   large_version         500 non-null    int64 
 8   split                 500 non-null    object
 9   product_title         500 non-null    object
 10  product_description   253 non-null    object
 11  product_bullet_point  456 non-null    object
 12  product_brand         486 non-null    object
 13  product_color         362 non-null    object
dtypes: int64(4), object(10)
memory usage: 54.8+ KB


In [13]:
df_query_filtered_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 528 entries, 25055 to 2246296
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            528 non-null    int64 
 1   query                 528 non-null    object
 2   query_id              528 non-null    int64 
 3   product_id            528 non-null    object
 4   product_locale        528 non-null    object
 5   esci_label            528 non-null    object
 6   small_version         528 non-null    int64 
 7   large_version         528 non-null    int64 
 8   split                 528 non-null    object
 9   product_title         528 non-null    object
 10  product_description   266 non-null    object
 11  product_bullet_point  480 non-null    object
 12  product_brand         514 non-null    object
 13  product_color         384 non-null    object
dtypes: int64(4), object(10)
memory usage: 61.9+ KB


In [14]:
# Save dfs to parquet and csv for checkpointing
df_sampled_rows.to_parquet("../data/processed/df_sampled_rows.parquet")
df_sampled_rows.to_csv("../data/processed/df_sampled_rows.csv", index=False)
df_query_filtered_rows.to_parquet("../data/processed/df_query_filtered_rows.parquet")
df_query_filtered_rows.to_csv("../data/processed/df_query_filtered_rows.csv", index=False)

In [17]:
print(f"Example queries:")
for query in df_sampled_rows['query'].unique()[:10]:
    print(f"- {query}")

Example queries:
- 10x mirror with lights without batteries
- alarm clock without snooze
- a/c adapters
- ken burns documentaries
- bridal shower food decorations
- stand up heaters for garage
- dive watches for men
- hi hand stmbol sticker
- punjabi salwar suits simple classy
- alexa devices


In [18]:
print(f"Example queries:")
for query in df_query_filtered_rows['query'].unique()[:10]:
    print(f"- {query}")

Example queries:
- 1 step motorhome cleaner without washing
- 10x mirror with lights without batteries
- 14 inch lamp shades for table lamps
- 24 inch deep pocket king sheets
- 400 micro sd card
- a/c adapters
- air zoom pegasus 35 running shoe
- alarm clock without snooze
- alexa devices
- art supplies for kids age 7-9
